Let me first summarize about the history of NLP, and the use of RNN. From what i understood about RNNs, they try to understand the semantic meaning of a sentence in a very different way than the current attention mechanism. While the current attention mechanism uses parallel computation, RNNs used sequential models to understand meaning. For each word that the RNN takes in, it alters the semantic meaning vector and incorporates what it understood by the addition of that word, now only the new modified vector is passed on to the next input token, so the RNN model doesn't have access to the previous meanings when decoding. Hence, the decoder only gets the end meaning of what the encoder thinks the meaning is, this is quite similar to what a human translator does, the person understands what the first person is saying completely, changing the meaning in their head as the person speaks each word and then translates the meaning in their mind and then says the whole sentence. The decoder only has the final semantic meaning vector available and undoes the process the encoder did. This method is only efficient for small sentences, for longer paragraphs it loses the wider sight and forgets the older context pieces. The addition of the Bahnman attention only gives the decoder access to the semantic vector corresponding to the token it is currently decoding, but still there needs to be sequential processing, which is quite inefficient. The thought of RNN is quiite intuitive and natural, it tries to mimick the real word translations but its inefficiencies overshadow its applicability. 

The attention mechanism on the other hand incorporates the Bahnuman attention with the original transformer, the theory behind attention blocks is very interesting and uses a lot of linear algebra. Imagine a high-dimensional vector space, each dimension in this vector space encloses a particular meaning or thought, for example, one dimension indicates gender, one dimension indicates fatness, etc. Each time a token is encountered, it is given a vector representation in this vector space based on what meaning it encloses, initially the vector might point in random direction but after training on data, it understands the semantic meaning of a word and what directions or dimension it should possess. The vector representation we created in the previous chapter, that is the tokens was the same thing, the vector representation of each token encloses its meaning in that high dimensional vector space. Now, the thing we use to eliminate the sequential problem with RNNs is to use scalar products. To understand the semantic relationship between two tokens, we compute the dot product, the more positive the dot product, the more similarity in the meaning of the tokens.This is a very powerful tool because dot products done for multiple vectors is just matrix multiplication which can be done quickly on a GPU. 

On the practical part, for each token we have a tensor representation. For each token we compute its dot product with each of the other tokens, these are the attention scores, then multiplying these attention scores with each of the inputs vectors, we get the context vectors for each input token. This is the implementation of the simple attention block without trainable weights.

In [41]:
import torch 

In [42]:
import torch

inputs = torch.tensor([
    [0.42, 0.81, 0.33, 0.50, 0.69],  # "with"
    [0.75, 0.60, 0.28, 0.79, 0.41],  # "great"
    [0.63, 0.44, 0.72, 0.37, 0.53],  # "power"
    [0.51, 0.36, 0.29, 0.65, 0.48],  # "comes"
    [0.39, 0.82, 0.41, 0.55, 0.72],  # "even"
    [0.78, 0.57, 0.33, 0.85, 0.46],  # "greater"
    [0.44, 0.49, 0.58, 0.63, 0.31],  # "responsibility"
])

query = inputs[5]

attn_score_5 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_score_5[i] = torch.dot(query, x_i)

print(attn_score_5)

tensor([1.6406, 1.8795, 1.5381, 1.4720, 1.7056, 1.9763, 1.4920])


These attention score are corresponding to the 5th token, similary attention score is computed for each of the token. The attention score is basically the dot product of the query vector with each of the other vectors.

In [43]:
attn_weights_5 = torch.softmax(attn_score_5, dim=0)
print(attn_weights_5)
print(attn_weights_5.sum())

tensor([0.1361, 0.1729, 0.1229, 0.1150, 0.1453, 0.1905, 0.1173])
tensor(1.0000)


To normalize all the elements of the vector we use the softmax function which normalizes all the elements and also makes them positive, the underlying function is exp(x)/ sum(exp(x_i)) for all i from 0 to n-1

In [44]:
context_vec_5 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_5 += attn_weights_5[i] * x_i

print(context_vec_5)

tensor([0.5798, 0.5947, 0.4056, 0.6406, 0.5137])


This was the context vector corresponding to only one input token, now computing all the context vectors together

In [45]:
attn_weights = torch.softmax(inputs @ inputs.T, dim=-1)
print(attn_weights)

tensor([[0.1644, 0.1493, 0.1270, 0.1091, 0.1759, 0.1601, 0.1141],
        [0.1381, 0.1725, 0.1221, 0.1152, 0.1464, 0.1880, 0.1177],
        [0.1412, 0.1467, 0.1588, 0.1126, 0.1526, 0.1606, 0.1276],
        [0.1404, 0.1602, 0.1303, 0.1231, 0.1488, 0.1739, 0.1232],
        [0.1642, 0.1476, 0.1281, 0.1080, 0.1776, 0.1594, 0.1152],
        [0.1361, 0.1729, 0.1229, 0.1150, 0.1453, 0.1905, 0.1173],
        [0.1386, 0.1546, 0.1394, 0.1164, 0.1500, 0.1676, 0.1333]])


In [46]:
context_vecs = attn_weights @ inputs
print(context_vecs)

tensor([[0.5604, 0.6094, 0.4103, 0.6228, 0.5301],
        [0.5786, 0.5954, 0.4055, 0.6399, 0.5144],
        [0.5677, 0.5919, 0.4242, 0.6192, 0.5190],
        [0.5719, 0.5930, 0.4106, 0.6323, 0.5161],
        [0.5597, 0.6097, 0.4113, 0.6220, 0.5304],
        [0.5798, 0.5947, 0.4056, 0.6406, 0.5137],
        [0.5693, 0.5922, 0.4173, 0.6277, 0.5152]])


This was just a simple implementation of the attention mechanism, now we will add trainable weights to this self-attention mechanism. For this we introduce three new matrices, they are key, query and value weights matrices. Using the vector embedding which we recieved after the chapter2 implementation, we apply matrix multiplication of each of the token vector with these key, query and value matrices to get the key, query and value vectors. For a particular query vector, to compute its attention we get the scalar product of all the key vectors of all the tokens with the query to get the attention score(w), then this attention scores are normalized by dividing by sq root of the number of dimensions. This normalization is carried out to improve the training performance by avoiding small gradients. Then we apply the softmax function on this to obtain the attention weights. Then these attention weights are multiplied with each and every value vector to get the context vector. This process is implemented below

In [47]:
x_5 = inputs[4]
d_in = inputs.shape[1]
d_out = 4
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

First let's compute the context vector with trainable weights for only one input token, for my case i took the 5th token. Later we can compute it for all of them.

In [48]:
query_5 = x_5 @ W_query
key_5 = x_5 @ W_key
value_5 = x_5 @ W_value

print(query_5)

tensor([0.3786, 1.9137, 0.7885, 1.3308])


In [49]:
keys = inputs @ W_key
values = inputs @ W_value
attn_scores_5 = query_5 @ keys.T
attn_scores_5

tensor([6.8032, 7.6202, 6.6862, 5.9020, 7.0950, 7.9952, 6.3249])

In [50]:
d_k = keys.shape[1]
attn_weight_5 = torch.softmax(attn_scores_5 / (d_k ** 0.5), dim=-1)
attn_weight_5

tensor([0.1275, 0.1918, 0.1202, 0.0812, 0.1475, 0.2314, 0.1004])

In [51]:
context_vectr_5 = attn_weight_5 @ values
context_vectr_5

tensor([1.4010, 1.7008, 1.3536, 1.2455])

This is how context vectors are defined with trainable weights, to do this for all tokens in a compact we will implement a python class

In [52]:
class Self_Attention_V1(torch.nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = torch.nn.Parameter(torch.rand(d_in, d_out))
        
    def forward(self, x):
        query = x @ W_query
        value = x @ W_value
        key = x @ W_key
        
        attn_scores = query @ key.T
        attn_wts = torch.softmax(attn_scores / key.shape[-1]**0.5, dim=-1)
        
        context_vec = attn_wts @ value
        return context_vec
    
torch.manual_seed(123)
sa_v1 = Self_Attention_V1(d_in, d_out)
print(sa_v1(inputs))

tensor([[1.3997, 1.6997, 1.3524, 1.2438],
        [1.4001, 1.6994, 1.3524, 1.2441],
        [1.4015, 1.7009, 1.3539, 1.2461],
        [1.3963, 1.6962, 1.3488, 1.2388],
        [1.4010, 1.7008, 1.3536, 1.2455],
        [1.4017, 1.7006, 1.3539, 1.2463],
        [1.3974, 1.6973, 1.3500, 1.2404]])


In [53]:
class Self_Attention_V2(torch.nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = Self_Attention_V2(d_in, d_out)
print(sa_v2(inputs))

tensor([[ 0.0333,  0.1708,  0.0535, -0.3977],
        [ 0.0337,  0.1710,  0.0533, -0.3973],
        [ 0.0333,  0.1712,  0.0527, -0.3969],
        [ 0.0334,  0.1710,  0.0533, -0.3974],
        [ 0.0333,  0.1708,  0.0535, -0.3976],
        [ 0.0338,  0.1710,  0.0533, -0.3973],
        [ 0.0331,  0.1711,  0.0530, -0.3972]], grad_fn=<MmBackward0>)


For both of these two versions of self-attention, the random weights with which the matrices are intialized are different so we are gaining different outputs at the end. If we use the same for both of them, then obviously the final context vectors would the be the same, the rest of the process is same. Linear provides robust, framework-optimized initialization, reducing the risk of poor convergence due to improper weight scaling. It also simplifies the inclusion and initialization of biases, streamlining model design.

Moving on to Casual Attention now, Casual attention is an alternative to self-attention which is used for specific tasks like next-word prediction and efficient output prediction. Casual attention masks out all the token which come after the query token for each of the tokens, so that in a sequence of 10 words, the 5th word would only have access to the previous 4 words, the next 5 words will be unavailable to it. This makes sure that data is not leaked, in other words this does not allow the model to know what words should come next and let it predict them itself. 

In [54]:
context_length = attn_scores_5.shape[0]
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores_5.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[6.8032,   -inf,   -inf,   -inf,   -inf,   -inf,   -inf],
        [6.8032, 7.6202,   -inf,   -inf,   -inf,   -inf,   -inf],
        [6.8032, 7.6202, 6.6862,   -inf,   -inf,   -inf,   -inf],
        [6.8032, 7.6202, 6.6862, 5.9020,   -inf,   -inf,   -inf],
        [6.8032, 7.6202, 6.6862, 5.9020, 7.0950,   -inf,   -inf],
        [6.8032, 7.6202, 6.6862, 5.9020, 7.0950, 7.9952,   -inf],
        [6.8032, 7.6202, 6.6862, 5.9020, 7.0950, 7.9952, 6.3249]])


Using this method is much better computationally rather than using the weights matrix which is already normalized by softmax, so using these attn scores and making the upper triangular part into all -inf we can get them as zero after softmax because softmax carries out exp of it. 

In [55]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3993, 0.6007, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2900, 0.4364, 0.2736, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2448, 0.3683, 0.2309, 0.1560, 0.0000, 0.0000, 0.0000],
        [0.1908, 0.2870, 0.1799, 0.1216, 0.2207, 0.0000, 0.0000],
        [0.1417, 0.2132, 0.1337, 0.0903, 0.1640, 0.2572, 0.0000],
        [0.1275, 0.1918, 0.1202, 0.0812, 0.1475, 0.2314, 0.1004]])


In addition to just masking, we also add dropout to our casual attention mechanism, which randomly selects a defined percentage of the data and masks it, i.e. the model is not allowed to access. So this allows us to reduce overfitting while training

In [56]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7986, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5801, 0.0000, 0.5471, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4618, 0.3120, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5740, 0.0000, 0.2431, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.1806, 0.3279, 0.5143, 0.0000],
        [0.0000, 0.3836, 0.0000, 0.0000, 0.2950, 0.0000, 0.0000]])


Now, combining the casual attention and dropout in a compact casual self-attention class

In [57]:
batch = torch.stack((inputs, inputs), dim=0)
batch

tensor([[[0.4200, 0.8100, 0.3300, 0.5000, 0.6900],
         [0.7500, 0.6000, 0.2800, 0.7900, 0.4100],
         [0.6300, 0.4400, 0.7200, 0.3700, 0.5300],
         [0.5100, 0.3600, 0.2900, 0.6500, 0.4800],
         [0.3900, 0.8200, 0.4100, 0.5500, 0.7200],
         [0.7800, 0.5700, 0.3300, 0.8500, 0.4600],
         [0.4400, 0.4900, 0.5800, 0.6300, 0.3100]],

        [[0.4200, 0.8100, 0.3300, 0.5000, 0.6900],
         [0.7500, 0.6000, 0.2800, 0.7900, 0.4100],
         [0.6300, 0.4400, 0.7200, 0.3700, 0.5300],
         [0.5100, 0.3600, 0.2900, 0.6500, 0.4800],
         [0.3900, 0.8200, 0.4100, 0.5500, 0.7200],
         [0.7800, 0.5700, 0.3300, 0.8500, 0.4600],
         [0.4400, 0.4900, 0.5800, 0.6300, 0.3100]]])

In [58]:
class CasualAttention(torch.nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = torch.nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias = qkv_bias)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))
        
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.transpose(1,2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )
        attn_weights = torch.softmax(attn_scores/ keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
context_length = batch.shape[1]
ca = CasualAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)
print(context_vecs)

tensor([[[-0.0553,  0.3606,  0.2267, -0.5092],
         [ 0.0213,  0.3872,  0.2719, -0.5039],
         [-0.0032,  0.3648,  0.2647, -0.4324],
         [ 0.0283,  0.3723,  0.2566, -0.4138],
         [ 0.0139,  0.3764,  0.2526, -0.4275],
         [ 0.0328,  0.3897,  0.2659, -0.4388],
         [ 0.0388,  0.3837,  0.2661, -0.4037]],

        [[-0.0553,  0.3606,  0.2267, -0.5092],
         [ 0.0213,  0.3872,  0.2719, -0.5039],
         [-0.0032,  0.3648,  0.2647, -0.4324],
         [ 0.0283,  0.3723,  0.2566, -0.4138],
         [ 0.0139,  0.3764,  0.2526, -0.4275],
         [ 0.0328,  0.3897,  0.2659, -0.4388],
         [ 0.0388,  0.3837,  0.2661, -0.4037]]], grad_fn=<UnsafeViewBackward0>)


Now we will create a multi-headed attention system.

In [59]:
class MultiHeadAttentionWrapper(torch.nn.Module):
    
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias = False):
        super().__init__()
        self.heads = torch.nn.ModuleList(
            [CasualAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )
    
    def forward(self, x) :
        return torch.cat([head(x) for head in self.heads], dim = -1)
    
torch.manual_seed(123)

context_length = batch.shape[1]
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=4)
context_vecs = mha(batch)
print(context_vecs)
print(context_vecs.shape)

tensor([[[-0.0553,  0.3606,  0.2267, -0.5092, -0.3257,  0.2227, -0.0507,
           0.1005,  0.7244,  0.7173, -0.4603, -0.2592, -0.2397, -0.4058,
          -0.1396, -0.0466],
         [ 0.0213,  0.3872,  0.2719, -0.5039, -0.2560,  0.2225,  0.0244,
           0.1121,  0.7013,  0.7570, -0.3735, -0.2065, -0.3265, -0.4187,
          -0.1197, -0.2136],
         [-0.0032,  0.3648,  0.2647, -0.4324, -0.1919,  0.2534,  0.0846,
           0.1336,  0.5840,  0.7838, -0.3495, -0.2304, -0.2703, -0.4720,
          -0.1734, -0.1965],
         [ 0.0283,  0.3723,  0.2566, -0.4138, -0.1828,  0.2436,  0.0838,
           0.1499,  0.5692,  0.7576, -0.3203, -0.1831, -0.2548, -0.4286,
          -0.1502, -0.2131],
         [ 0.0139,  0.3764,  0.2526, -0.4275, -0.2175,  0.2490,  0.0568,
           0.1434,  0.5992,  0.7590, -0.3483, -0.1950, -0.2483, -0.4254,
          -0.1512, -0.1820],
         [ 0.0328,  0.3897,  0.2659, -0.4388, -0.2119,  0.2493,  0.0675,
           0.1472,  0.6162,  0.7758, -0.3395, -0.184

In [60]:
import torch.nn as nn
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias = False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out) 
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )
        
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        
        attn_scores = queries @ keys.transpose(2,3)
        
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) 

        return context_vec
    
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)

tensor([[[ 0.4139, -0.4688,  0.0254,  0.2682],
         [ 0.4069, -0.4891,  0.0168,  0.2405],
         [ 0.3752, -0.4877,  0.0420,  0.2574],
         [ 0.3785, -0.4938,  0.0304,  0.2417],
         [ 0.3853, -0.4897,  0.0328,  0.2443],
         [ 0.3901, -0.4941,  0.0288,  0.2348],
         [ 0.3756, -0.4974,  0.0360,  0.2355]],

        [[ 0.4139, -0.4688,  0.0254,  0.2682],
         [ 0.4069, -0.4891,  0.0168,  0.2405],
         [ 0.3752, -0.4877,  0.0420,  0.2574],
         [ 0.3785, -0.4938,  0.0304,  0.2417],
         [ 0.3853, -0.4897,  0.0328,  0.2443],
         [ 0.3901, -0.4941,  0.0288,  0.2348],
         [ 0.3756, -0.4974,  0.0360,  0.2355]]], grad_fn=<ViewBackward0>)
